# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [1]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    # TODO: implement function
    # read file
    edges = dict()
    demand = dict()
    for row in open(infile):
        row = row.strip().split(' ')
        if row[0] == 'p':
            n_nodes = int(row[2])
        if row[0] == 'n':
            demand[int(row[1])] = int(row[2])
        if row[0] == 'a':
            edge = (int(row[1]), int(row[2]))
            if edge in edges.keys():
                edges[edge]['capacity'].append(int(row[4]))
                edges[edge]['weight'].append(int(row[5]))               
            else:
                attr = dict()
                attr['capacity'] = [int(row[4])]
                attr['weight'] = [int(row[5])]
                edges[edge] = attr
    # add new nodes to solve multi-edges problems        
    for edge in edges.keys():
        if len(edges[edge]['capacity']) > 1:
            for i in range(len(edges[edge]['capacity'])):
                n_nodes += 1
    # add a node between one edge, then add a new edge             
                edge1 = (edge[0],n_nodes)
                attr = dict()
                attr['capacity'] = edges[edge]['capacity'][i]
                attr['weight'] = 1/2. * edges[edge]['weight'][i]
                edges[edge1] = attr
    # add another edge            
                edge2 = (n_nodes, edge[1])
                attr = dict()
                attr['capacity'] = edges[edge]['capacity'][i]
                attr['weight'] = 1/2. * edges[edge]['weight'][i]
                edges[edge2] = attr
    # delete the original edge
            del edges[edge]
            
    G = nx.DiGraph()

    for edge in edges.keys():
        G.add_edge(edge[0], edge[1])
    for (u, v) in G.edges():
        G.edge[u][v]['capacity'] = edges[(u, v)]['capacity']
        G.edge[u][v]['weight'] = edges[(u, v)]['weight']
    for node in demand:
        G.node[node]['demand'] = demand[node]   
    
    return G
    
    pass

The following will check that your code outputs the expected min cost flow values on several test instances.

In [2]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [7]:
import pulp

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    # TODO: implement function
    Prob = pulp.LpProblem("minimum_cost_flow", pulp.LpMinimize)
    cost = dict()

    for edge in G.edges():
        cost[str(edge)] = G.get_edge_data(edge[0], edge[1])['weight']

    flow = pulp.LpVariable.dict("flow", [str(edge) for edge in G.edges()], 0)

    for edge in G.edges():
        Prob += flow[str(edge)] <= G.get_edge_data(edge[0], edge[1])['capacity']

    for node in G.nodes():
        if G.node[node] == {}:
            demand = 0
        else:
            demand = G.node[node]['demand']     
        f_in = []
        f_out = []
        for edge in G.edges():
            if node == edge[1]:
                f_in.append(flow[str(edge)])
            if node == edge[0]:
                f_out.append(flow[str(edge)])            
        Prob += pulp.lpSum(f_in) - pulp.lpSum(f_out) == demand

    Prob += pulp.lpSum([cost[str((u,v))]*flow[str((u,v))] for (u,v) in G.edges()])
    status = Prob.solve()
    
    return int(pulp.value(Prob.objective))    
    
    pass

The following will check that the LP finds the same optimal values as previously.

In [8]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
